In [1]:
from tqdm.auto import tqdm 
import pandas as pd
import numpy as np
import xarray as xr
import netCDF4 as nf
from netCDF4 import Dataset
%matplotlib inline
import glob
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [2]:
datapath="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/Data/ECMWF/ERA5_25kmx3hr/"
path="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/besttracks/"
output="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/create_ts/outputs/"
target="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/create_ts/outputs/targets/"

In [106]:
track = sorted(glob.glob(path+'*2010*.csv'))

In [107]:
tracksDF = pd.read_csv(track[0])
tracksDF['name'].unique()

array(['OMAIS', 'CONSON', 'CHANTHU', 'DIANMU', 'LIONROCK', 'KOMPASU',
       'MALOU', 'MERANTI', 'FANAPI', 'MALAKAS', 'MEGI', 'CHABA', 'OMEKA'],
      dtype=object)

In [108]:
stormnames = list(tracksDF['name'].unique())

In [109]:
[str(tracksDF[tracksDF['name']==stormnames[0]].time[i]).split(':')[0] for i in range(len(tracksDF[tracksDF['name']==stormnames[0]].time))]

['2010-03-19',
 '2010-03-19',
 '2010-03-19',
 '2010-03-19',
 '2010-03-19',
 '2010-03-19',
 '2010-03-20',
 '2010-03-20',
 '2010-03-20',
 '2010-03-20',
 '2010-03-20',
 '2010-03-20',
 '2010-03-20',
 '2010-03-20',
 '2010-03-21',
 '2010-03-21',
 '2010-03-21',
 '2010-03-21',
 '2010-03-21',
 '2010-03-21',
 '2010-03-21',
 '2010-03-21',
 '2010-03-22',
 '2010-03-22',
 '2010-03-22',
 '2010-03-22',
 '2010-03-22',
 '2010-03-22',
 '2010-03-22',
 '2010-03-22',
 '2010-03-23',
 '2010-03-23',
 '2010-03-23',
 '2010-03-23',
 '2010-03-23',
 '2010-03-23',
 '2010-03-23',
 '2010-03-23',
 '2010-03-24',
 '2010-03-24',
 '2010-03-24',
 '2010-03-24',
 '2010-03-24',
 '2010-03-24',
 '2010-03-24',
 '2010-03-24',
 '2010-03-25',
 '2010-03-25',
 '2010-03-25',
 '2010-03-25',
 '2010-03-25',
 '2010-03-25',
 '2010-03-25',
 '2010-03-25',
 '2010-03-26',
 '2010-03-26',
 '2010-03-26',
 '2010-03-26',
 '2010-03-26',
 '2010-03-26',
 '2010-03-26']

In [110]:
def largearea_withpres(dataset=None,invar=None,indices=None,tc_irad=None):
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","plev","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices))),
               plev=(["plev"],dataset.plev.data)),
    attrs=dict(description="coords with matrices"),)
    
    LATN,LATS,LONE,LONW = tc_irad[0,:]
    testsmall = ds['variable'][0,0,:,:].sel(lat=slice(LATS,LATN),lon=slice(LONE,LONW))
    if testsmall.shape[0]<testsmall.shape[1]:
        rgspt = int(testsmall.shape[0])
    else:
        rgspt = int(testsmall.shape[1])
    var_out=np.zeros((len(indices),len(dataset.plev.data),rgspt,rgspt))
    del testsmall
    
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_irad[it,:]
        for ip in range(len(dm1.plev.data)):
            try:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
            except:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:rgspt,0:rgspt]
    return var_out  

In [111]:

def output_indices(TCtrack=None,ERA5date=None,ERA5hour=None):
    allindices = []
    for timeidx in range(len(TCtrack)):#len(track['time'])):
        datetrack,hourtrack = TCtrack['time'][timeidx].split(':')[0],TCtrack['time'][timeidx].split(':')[1][0:2]
        ####################################################################################################
        # Find the indices in ERA5 data with the same date as track
        ####################################################################################################
        dateind = []
        for ind,obj in enumerate(ERA5date):
            if obj==datetrack:
                dateind.append(ind)
        del ind,obj
        hourind = []
        hourextract = ERA5hour[int(np.min(np.asarray(dateind))):int(np.max(np.asarray(dateind)))+1]
        for ind,obj in enumerate(hourextract):
            if obj==hourtrack:            
                hourind.append(ind)
        allindices.append((int(np.min(np.asarray(dateind))),int(hourind[0])))
    return allindices

def extract_var(dataset=None,var='var138',indices=None):
    extractedvar = []
    for i in (range(len(indices))):
        realindex = indices[i][0]+indices[i][1]
        extractedvar.append(dataset[var][int(realindex),...].data)
    return np.asarray(extractedvar)

def smallarea(dataset=None,invar=None,indices=None,tc_irad=None):
    if len(invar.shape) != 3:
        invar = np.squeeze(invar)
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices)))),
    attrs=dict(description="coords with matrices"),)
    
    LATN,LATS,LONE,LONW = tc_irad[0,:]
    testsmall = ds['variable'][0,:,:].sel(lat=slice(LATS,LATN),lon=slice(LONE,LONW))
    if testsmall.shape[0]<testsmall.shape[1]:
        rgspt = int(testsmall.shape[0])
    else:
        rgspt = int(testsmall.shape[1])
    rgspt=16
    var_out=np.zeros((len(indices),rgspt,rgspt))
    del testsmall
    
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_irad[it,:]
        try:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
        except:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:rgspt,0:rgspt]
    return var_out




In [112]:
#for dm1 files functions
def make_timeseries_step1(TCname=None,era5_date=None,era5_hour=None):
    track=tracksDF[tracksDF['name']==TCname].reset_index()
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    return pos,indices_store,tc_irad

def make_timeseries_step3(dm1=None,pvarname=None,pvardict=None,indices_store=None):
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...].flatten() for i in range(len(indices_store))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname[ind]] = np.asarray(pvarTS_store).transpose()
    return ts_pdict

In [113]:
def readyear_automatic(year=None):
    dm2 = xr.open_mfdataset([datapath+'/eqt/eqt_200_'+str(year)+'.nc',datapath+'/eqt/eqt_250_'+str(year)+'.nc',\
                             datapath+'/eqt/eqt_300_'+str(year)+'.nc',datapath+'/eqt/eqt_400_'+str(year)+'.nc',\
                             datapath+'/eqt/eqt_500_'+str(year)+'.nc',datapath+'/eqt/eqt_600_'+str(year)+'.nc',\
                             datapath+'/eqt/eqt_700_'+str(year)+'.nc',datapath+'/eqt/eqt_800_'+str(year)+'.nc',\
                             datapath+'/eqt/eqt_850_'+str(year)+'.nc',datapath+'/eqt/eqt_1000_'+str(year)+'.nc',\
                             datapath+'/eqt/eqt_925_'+str(year)+'.nc'])
    #tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/besttracks/nio/*_'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,era5_date,era5_hour

dm2,era5_date,era5_hour = readyear_automatic(2010)

In [114]:
TCeq_ts = []
for TCname in tqdm(stormnames):
    ###########################################################################
    pos,indices_store,tc_irad = make_timeseries_step1(TCname=TCname,era5_date=era5_date,era5_hour=era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    smallsvarout = [smallarea(dm2,mysvar[i],indices_store,tc_irad) for i in (range(len(mysvar)))]
    
    svarname1 = ['eqt1000','eqt200','eqt250','eqt300','eqt400',\
                 'eqt500','eqt600','eqt700','eqt800',\
                 'eqt850','eqt925']
   
    svarname = ['eqt1000','eqt200','eqt250','eqt300','eqt400',\
                'eqt500','eqt600','eqt700','eqt800',\
                'eqt850','eqt925']
    
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    ##################################################################################################
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...].flatten() for i in range(len(indices_store))]
        tsdict[svarname[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCeq_ts.append(tsdict)

  0%|          | 0/13 [00:00<?, ?it/s]

In [115]:
storeTCdicts = {}
for ind in range(len(TCeq_ts)):
    storeTCdicts[stormnames[ind]] = pd.DataFrame.from_dict(TCeq_ts[ind])

In [116]:
for ind,obj in enumerate(stormnames):
    storeTCdicts[obj].to_csv(output+'2010_eqtlevs_wpac_'+str(obj)+'.csv')

In [147]:
###############################read year from data and track ####################################

def readyear(year=None):
    dm2 = xr.open_mfdataset([datapath+'/eqt/eqt_200_'+str(year)+'.nc',datapath+'/eqt/eqt_250_'+str(year)+'.nc',\
                             datapath+'/eqt/eqt_300_'+str(year)+'.nc',datapath+'/eqt/eqt_400_'+str(year)+'.nc',\
                             datapath+'/eqt/eqt_500_'+str(year)+'.nc',datapath+'/eqt/eqt_600_'+str(year)+'.nc',\
                             datapath+'/eqt/eqt_700_'+str(year)+'.nc',datapath+'/eqt/eqt_800_'+str(year)+'.nc',\
                             datapath+'/eqt/eqt_850_'+str(year)+'.nc',datapath+'/eqt/eqt_1000_'+str(year)+'.nc',\
                             datapath+'/eqt/eqt_925_'+str(year)+'.nc'])
    tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/besttracks/wp/*_'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,tracklist,era5_date,era5_hour

dm2,tracklist,era5_date,era5_hour = readyear(2010)

TCeqt_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    lonx=np.mod(lon1,360)
    pos = arr = np.stack((lat1, lonx), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    ################################################################################################
    
    smallsvarout = [smallarea(dm2,mysvar[i],indices_store,tc_irad) for i in (range(len(mysvar)))]
    
    svarname1 = ['eqt1000','eqt200','eqt250','eqt300','eqt400',\
                 'eqt500','eqt600','eqt700','eqt800',\
                 'eqt850','eqt925']
   
    svarname = ['eqt1000','eqt200','eqt250','eqt300','eqt400',\
                'eqt500','eqt600','eqt700','eqt800',\
                'eqt850','eqt925']
    
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    ##################################################################################################
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...].flatten() for i in range(len(indices_store))]
        tsdict[svarname[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCeqt_ts.append(tsdict)

  0%|          | 0/1 [00:00<?, ?it/s]

In [148]:
dm1=xr.open_dataset(datapath+'/div/div_2010.nc')

In [149]:
#divergence
TCd_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2 
    
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store,tc_irad) for i in (range(len(mypvar)))]    
    pvarname = ['indiv']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...].flatten() for i in range(len(indices_store))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCd_ts.append(ts_pdict)

  0%|          | 0/1 [00:00<?, ?it/s]

In [150]:
plv=[str(int(obj)) for obj in dm1.plev.data/100]

In [151]:
plv

['50',
 '100',
 '200',
 '250',
 '300',
 '400',
 '500',
 '600',
 '700',
 '800',
 '850',
 '925',
 '1000']

In [153]:
tracklist

['/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/besttracks/wp/nwp_20101011.csv']

In [154]:
tempdivdict0 = {'div_'+plv[i]:TCd_ts[0]['indiv'][:,i] for i in range(13)}
#tempdivdict1 = {'div_'+plv[i]:TCd_ts[1]['indiv'][:,i] for i in range(13)}
#tempdivdict2 = {'div_'+plv[i]:TCd_ts[2]['indiv'][:,i] for i in range(13)}
#tempdivdict3 = {'div_'+plv[i]:TCd_ts[3]['indiv'][:,i] for i in range(13)}
#tempdivdict4 = {'div_'+plv[i]:TCd_ts[4]['indiv'][:,i] for i in range(13)}
#tempdivdict5 = {'div_'+plv[i]:TCd_ts[5]['indiv'][:,i] for i in range(13)}
#tempdivdict6 = {'div_'+plv[i]:TCd_ts[6]['indiv'][:,i] for i in range(13)}
#tempdivdict7 = {'div_'+plv[i]:TCd_ts[7]['indiv'][:,i] for i in range(13)}

In [155]:
div1=pd.DataFrame.from_dict(tempdivdict0)
#div2=pd.DataFrame.from_dict(tempdivdict1)
#div3=pd.DataFrame.from_dict(tempdivdict2)
#div4=pd.DataFrame.from_dict(tempdivdict3)
#div5=pd.DataFrame.from_dict(tempdivdict4)
#div6=pd.DataFrame.from_dict(tempdivdict5)
#div7=pd.DataFrame.from_dict(tempdivdict6)
#div8=pd.DataFrame.from_dict(tempdivdict7)

In [156]:
tracklist

['/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/besttracks/wp/nwp_20101011.csv']

In [158]:
div1.to_csv(output+'inner/innerdiv_wp_20101011.csv')
#div2.to_csv(output+'inner/innerdiv_wp_20100616.csv')
#div3.to_csv(output+'inner/innerdiv_wp_20110926.csv')
#div4.to_csv(output+'inner/innerdiv_wp_20111211.csv')
#div5.to_csv(output+'inner/innerdiv_wp_20121125.csv')
#div6.to_csv(output+'inner/innerdiv_wp_20131102.csv')
#div7.to_csv(output+'inner/innerdiv_wp_20191117.csv')
#div8.to_csv(output+'inner/innerdiv_wp_20191124.csv')

In [159]:
eqt1=pd.DataFrame.from_dict(TCeqt_ts[0])
#eqt2=pd.DataFrame.from_dict(TCeqt_ts[1])
#eqt3=pd.DataFrame.from_dict(TCeqt_ts[2])
#eqt4=pd.DataFrame.from_dict(TCeqt_ts[3])
#eqt5=pd.DataFrame.from_dict(TCeqt_ts[4])
#eqt6=pd.DataFrame.from_dict(TCeqt_ts[5])
#eqt7=pd.DataFrame.from_dict(TCeqt_ts[6])
#eqt8=pd.DataFrame.from_dict(TCeqt_ts[7])

In [160]:
eqt1.to_csv(output+'inner/innereqt_wp_20101011.csv')
#eqt2.to_csv(output+'inner/innereqt_wp_20100616.csv')
#eqt3.to_csv(output+'inner/innereqt_wp_20110926.csv')
#eqt4.to_csv(output+'inner/innereqt_wp_20111211.csv')
#eqt5.to_csv(output+'inner/innereqt_wp_20121125.csv')
#eqt6.to_csv(output+'inner/innereqt_wp_20131102.csv')
#eqt7.to_csv(output+'inner/innereqt_wp_20191117.csv')
#eqt8.to_csv(output+'inner/innereqt_wp_20191124.csv')

In [111]:
eqt6

,eqt1000,eqt200,eqt250,eqt300,eqt400,eqt500,eqt600,eqt700,eqt800,eqt850,eqt925
0,345.297227,348.270750,347.172681,346.667049,342.538762,340.958077,338.261174,336.013414,337.599358,338.564100,342.240834
1,345.404039,347.582237,346.547803,346.200690,342.470740,341.012447,338.827018,336.408796,337.634005,338.544349,342.080728
2,343.893973,348.626453,347.416623,346.606991,343.100311,340.992425,339.235265,338.216541,338.383987,338.805238,342.056585
3,343.875684,349.692531,347.839921,346.539134,342.941311,341.469536,338.640718,337.712634,338.464624,338.911265,341.871960
4,343.404208,349.662110,348.229155,346.516916,342.986181,341.708513,338.154072,337.445915,338.859037,339.252007,341.427741
...,...,...,...,...,...,...,...,...,...,...,...
70,338.308408,353.001054,352.629395,352.133887,349.380954,346.042173,343.289340,343.403407,339.970359,337.966971,335.629743
71,336.626868,353.328028,352.418021,351.633981,348.456155,345.158419,342.803480,341.272717,337.497589,335.984824,333.836681
72,333.921793,353.985264,352.353518,351.090638,348.069929,344.588316,341.363290,339.435542,336.184182,334.110592,331.596682
73,332.542694,353.513694,351.352678,349.897197,346.936182,343.418656,340.573204,339.190121,336.340228,333.781755,330.562734


In [44]:
div_21=pd.DataFrame.from_dict(tempdivdict0)
div_21.to_csv(output+'inner/innerdiv_nio_20200514.csv')

In [45]:
div_21

,div_50,div_100,div_200,div_250,div_300,div_400,div_500,div_600,div_700,div_800,div_850,div_925,div_1000
0,-5.468071e-06,8.189818e-07,1.473643e-05,4.152243e-07,-1.595472e-07,-3.117195e-06,-1.406611e-05,-4.794681e-06,5.344860e-06,7.555704e-06,4.714704e-06,-0.000004,-0.000006
1,-1.470877e-05,-4.905509e-06,1.145946e-05,-2.617482e-06,9.514624e-07,-4.295493e-06,-9.086158e-06,2.373825e-06,5.397946e-06,4.018075e-06,3.169524e-06,-0.000006,-0.000010
2,1.896304e-06,-4.935544e-06,2.680063e-05,-3.260328e-06,-5.279086e-06,-2.325571e-06,-1.981098e-06,4.870933e-06,-1.163397e-06,2.354034e-06,2.502929e-08,-0.000013,-0.000012
3,9.262585e-07,-2.962770e-08,3.071874e-05,1.278915e-05,2.805842e-06,-8.845236e-06,-4.142290e-06,3.840192e-06,-3.946712e-06,-1.601758e-06,-2.710265e-06,-0.000012,-0.000009
4,3.901499e-06,-6.601622e-06,8.415605e-06,6.219023e-06,1.322478e-07,-1.331192e-05,-5.270878e-06,4.022149e-08,6.560353e-06,5.247421e-06,1.549721e-06,-0.000004,-0.000004
5,9.145006e-07,-1.049298e-05,1.471199e-05,-3.112727e-06,-9.519048e-06,-3.787194e-06,1.970853e-06,8.612988e-07,5.974725e-06,1.789420e-06,-1.547975e-06,-0.000009,-0.000012
6,9.392970e-07,-1.301174e-05,4.277099e-07,-1.040264e-05,-1.470110e-05,3.484776e-06,2.608402e-06,3.929832e-06,2.482207e-06,8.658972e-07,-7.150229e-07,-0.000007,-0.000010
7,-6.796967e-06,-4.856498e-06,9.957002e-06,-1.804437e-06,-1.211104e-05,3.747933e-06,4.718138e-06,6.890506e-06,-5.134789e-06,-7.256051e-06,-6.525428e-06,-0.000010,-0.000012
8,3.125853e-06,-9.771204e-06,6.327289e-06,6.208429e-06,-3.935711e-07,-6.589282e-06,-8.736970e-07,2.955901e-06,-1.372071e-06,2.710731e-07,-1.178007e-06,-0.000009,-0.000009
9,-4.170302e-06,6.722403e-06,5.595619e-06,1.370138e-05,1.127308e-06,-1.038524e-05,1.173990e-06,5.009526e-06,-1.864915e-06,-3.193156e-06,-4.992995e-06,-0.000012,-0.000012


In [29]:
#TCp_ts[0]['vort']stands for the vorticity of the first storm and vort again contains 21 pressure levels.
#preslv = [str(int(obj)) for obj in dm1.plev.data/100]
tempvortdict = {'vort_'+plev[i]:TCv_ts[0]['vort'][:,i] for i in range(21)}
temprhumdict = {'rhum_'+plev[i]:TCv_ts[0]['rhum'][:,i] for i in range(21)}
temppvordict = {'pvor_'+preslv[i]:TCp2_ts[0]['pvor'][:,i] for i in range(18)}
tempgpotdict = {'gpot_'+plv[i]:TCgp_ts[0]['gpot'][:,i] for i in range(20)}
temptempdict = {'temp_'+plev[i]:TCtl_ts[0]['temp'][:,i] for i in range(21)}
tempvveldict = {'vvel_'+preslv[i]:TCvv_ts[0]['vvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict2 = {**temppvordict,**tempgpotdict}
dict3 = {**temptempdict,**tempvveldict}
combine1={**dict1,**dict2}
alldict = {**combine1,**dict3}
#pts2020_amphan=pd.DataFrame.from_dict(plvdict)

In [31]:
plev_2021=pd.DataFrame.from_dict(alldict)

In [67]:
ts_gul21=pd.concat([div_21,eqt_21,sl2021,rad2021,pr2021,plev_2021], axis=1, join='inner')

In [68]:
ts_gul21

,div_50,div_100,div_200,div_250,div_300,div_400,div_500,div_600,div_700,div_800,...,vvel_250,vvel_300,vvel_400,vvel_500,vvel_600,vvel_700,vvel_850,vvel_925,vvel_975,vvel_1000
0,-1.470139e-05,-2.004806e-05,0.000018,1.717859e-05,1.207291e-05,8.906121e-06,-1.034816e-06,7.125200e-07,-3.069406e-06,-4.832400e-06,...,-0.179325,-0.259124,-0.360181,-0.382818,-0.385455,-0.363965,-0.295342,-0.182406,-0.074252,-0.017509
1,-2.488989e-05,-5.881069e-06,0.000022,1.001521e-05,7.239578e-06,-2.546585e-06,2.435117e-06,-5.611917e-06,-5.518668e-07,-1.899782e-06,...,-0.201773,-0.238356,-0.244928,-0.229633,-0.239007,-0.181803,-0.169252,-0.119534,-0.056306,-0.018626
2,-5.774957e-06,-1.693203e-06,0.000016,3.927038e-06,-8.301693e-06,-1.514552e-05,-1.117820e-06,2.739776e-06,-5.193695e-06,6.410002e-06,...,-0.151920,-0.137368,-0.016568,0.070683,0.044295,0.076153,0.026839,-0.005580,0.007360,0.018210
3,-4.540430e-06,5.546259e-06,0.000010,9.460666e-06,-8.159375e-06,-6.259594e-06,-4.779082e-06,1.751061e-06,-1.289300e-07,5.345559e-06,...,-0.156391,-0.158884,-0.086677,-0.047792,-0.028524,-0.031328,-0.089968,-0.073317,-0.020160,0.013664
4,1.277926e-05,2.870103e-06,0.000009,5.408539e-06,-6.243528e-06,6.831135e-06,1.678709e-07,1.325330e-06,7.220078e-07,1.140637e-06,...,-0.165562,-0.165935,-0.179049,-0.213646,-0.220336,-0.232739,-0.245793,-0.176871,-0.078091,-0.024020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,-3.872090e-06,-4.878500e-06,0.000037,4.795962e-06,8.417992e-07,4.533329e-06,-4.068715e-07,-1.222652e-05,-2.529082e-05,-1.379685e-05,...,-0.332974,-0.345632,-0.376871,-0.417450,-0.363439,-0.170226,0.073287,0.059968,0.022831,0.031646
77,1.046498e-05,-6.112386e-07,0.000041,8.770730e-07,-3.539375e-06,3.801601e-06,-2.524524e-06,-1.678348e-05,-3.209530e-05,-1.239870e-05,...,-0.463555,-0.455450,-0.437432,-0.476495,-0.402014,-0.120978,0.123721,0.091201,0.027067,0.026226
78,-3.096648e-07,-5.463196e-06,0.000052,1.734338e-05,-3.813999e-06,3.948808e-06,-9.298674e-06,-3.250362e-05,-2.863479e-05,-9.745010e-06,...,-0.633262,-0.663047,-0.619403,-0.636304,-0.430235,-0.075319,0.126537,0.079193,0.016556,0.001574
79,3.144261e-06,-1.742784e-05,0.000044,1.299020e-05,-1.123449e-05,-4.185305e-06,-1.553120e-05,-2.349971e-05,-1.744000e-05,4.005851e-07,...,-0.388316,-0.391463,-0.256868,-0.165840,0.070560,0.315451,0.356788,0.221411,0.084055,0.041359


In [69]:
ts_gul21.to_csv(output+'inner/inner_nio_20210924.csv')